# PCR report Work In Progress :)

In [ ]:
from pcrep import parse_input

input = '230901_GN004308-086_20230901_112734_186.csv'
dc = parse_input.parse_inputname(input)
dc

In [ ]:
# 230901_GN004308-086.md
out_name = dc['date'] + dc['gn'] + '.md'
out_name

In [ ]:
from pcrep.pcrep import well2idx
wrc = well2idx('A02')
print(wrc)

In [ ]:
import pandas as pd

df = pd.read_csv(
    "./example/230901_GN004308-086/230901_GN004308-086_20230901_112734_186.csv", delimiter=';')
df = df.replace(',', '.', regex=True)
CONC_NAME = 'Conc(copies/µL)'
df[CONC_NAME] = df[CONC_NAME].astype('Float64')
# df.info()

DIL_FINAL_FACTOR_NAME = 'final dilution factor'
DIL_SAMPLE_DESCRIPTION_NAME = 'Sample name'

df_conc = pd.read_csv(
    "./example/230901_GN004308-086/230901_GN004308-086_conc.csv", header=None)

In [ ]:
df_conc.index = range(1, len(df_conc) + 1)
# df_conc[CONC_SAMPLE_NAME] = df_conc.index

df_conc.columns = [DIL_SAMPLE_DESCRIPTION_NAME, DIL_FINAL_FACTOR_NAME]
df_conc

In [ ]:
df[["wr", "wc"]] = df.apply(lambda x: well2idx(
    x['Well']), axis='columns', result_type='expand')
df.set_index(['wr', 'wc'], inplace=True)

In [ ]:
df

In [ ]:
FDL_NAME = "final dilution factor"
SAMPLE_NAME = "Sample"
SAMPLE_NUM_NAME = "Sample description 1"

df.loc[:, [FDL_NAME]] = df[SAMPLE_NUM_NAME].map(
    df_conc[DIL_FINAL_FACTOR_NAME], na_action='ignore')

df.loc[:, [SAMPLE_NAME]] = df[SAMPLE_NUM_NAME].map(
    df_conc[DIL_SAMPLE_DESCRIPTION_NAME], na_action='ignore')

In [ ]:
def result_fn(conc, dil, a=20.0, b=2.0):
    """Compute results

    Parameters:
    conc : float
    dil : float
        final dilution factor of the sample
    a : float
        ddPCR Volume 20 µL
    b : float
        Sample volume in the ddPCR reaction 2 µL
    """
    return ((a * conc) * (1000.0 / b)) * dil


WELL_RESULT_NAME = 'vg/ml'
df.loc[:, [WELL_RESULT_NAME]] = df.apply(lambda x: result_fn(
    x['Conc(copies/µL)'], x['final dilution factor']), axis=1)

In [ ]:
df['Target'].unique()

In [ ]:
df_idt = df.query("Target == 'IDT'")
# df_idt

targets = df['Target'].unique()
dc = dict.fromkeys(targets)
for t in targets:
    dc[t] = df.query("Target == @t")

# dc['ITR']

In [ ]:
dc['ITR'].head()

In [ ]:
df_itr = dc['ITR']

In [ ]:
df_itr.head()

In [ ]:
def get_sample(dfa, sample_num):
    # TODO: check for valid `type` `and sample_num`
    dfa = dfa.loc[dfa['Sample description 1'] == sample_num]
    return dfa

In [ ]:
s = get_sample(dc['ITR'], 22)
s

In [ ]:
def process_sample(s):
    m = s.loc[:, WELL_RESULT_NAME].mean()
    t = s.loc[:, WELL_RESULT_NAME].std(ddof=0)
    if isinstance(m, float) and m != 0.0:
        cv = t / m
    else:
        cv = float("nan")
    return {'id': s['Sample description 1'][0], 'name': s[SAMPLE_NAME][0], 'mean': m, 'std': t, 'cv': cv}


process_sample(s)

In [ ]:
import json

params_file = './data/params.json'
with open(params_file) as json_file:
    check_params = json.load(json_file)
print(check_params)


def check_sample(sample, ref):
    cv_hreshold = ref['assesmentCriteria']['cvThreshold']
    passed = True
    comment = None
    if sample['cv'] > cv_hreshold:
        passed = False
        comment = "CV {:.2f} > {:.2f}".format(sample['cv'], cv_hreshold)
    return {"valid": passed, "comment": comment}


check_sample(process_sample(s), check_params)

In [ ]:
def gen_sample_results(df):
    targets = df['Target'].unique()
    df_res = pd.DataFrame(columns=['id', 'target', 'sample name',
                          'result [vg/ml]', 'STDE', 'CV [%]', 'comment', 'Valid', 'info'])
    for t in targets:
        dft = df.query("Target == @t")
        samples = dft[SAMPLE_NUM_NAME].unique()
        for n in samples:
            r = process_sample(get_sample(dft, n))
            check = check_sample(r, check_params)
            df_res.loc[len(df_res)] = [r['id'], t, r['name'],
                                       r['mean'], r['std'], r['cv'],
                                       check['comment'], check['valid'], None]

    return df_res


df_t = gen_sample_results(df)
df_t